C:\Users\Suresh Krishna\AppData\Local\Temp\ipykernel_27468\2342971961.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[num_cols] = num_imputer.fit_transform(X[num_cols])
C:\Users\Suresh Krishna\AppData\Local\Temp\ipykernel_27468\2342971961.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])
C:\Users\Suresh Krishna\AppData\Local\Temp\ipykernel_27468\2342971961.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

Random Forest:
               precision    recall  f1-score   support

    Critical       0.34      0.74      0.47       541
        High       0.71      0.88      0.79    127806
         Low       1.00      1.00      1.00    990925
      Medium       0.98      0.95      0.97    880728

    accuracy                           0.97   2000000
   macro avg       0.76      0.89      0.81   2000000
weighted avg       0.97      0.97      0.97   2000000

✅ Model and encoders saved (random_forest_risk_model.pkl, encoders.pkl)


In [2]:
!pip install numpy==1.26.0
!pip install scikit-learn==1.7.1
!pip install pandas joblib


  Using cached numpy-1.26.0-cp310-cp310-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.0-cp310-cp310-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2


  You can safely remove it manually.


  Using cached scikit_learn-1.7.1-cp310-cp310-win_amd64.whl.metadata (11 kB)
Using cached scikit_learn-1.7.1-cp310-cp310-win_amd64.whl (8.9 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.1
    Uninstalling scikit-learn-1.2.1:
      Successfully uninstalled scikit-learn-1.2.1


In [3]:
import numpy, sklearn, pandas
print(numpy.__version__)
print(sklearn.__version__)
print(pandas.__version__)


1.26.0
1.7.1
2.3.2


In [8]:
!pip install dask[complete] pandas scikit-learn joblib


  Using cached distributed-2025.7.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached bokeh-3.8.0-py3-none-any.whl.metadata (10 kB)
Using cached distributed-2025.7.0-py3-none-any.whl (1.0 MB)
Using cached bokeh-3.8.0-py3-none-any.whl (7.2 MB)

   ---------------------------------------- 0/2 [distributed]
   ---------------------------------------- 0/2 [distributed]
   ---------------------------------------- 0/2 [distributed]
   ---------------------------------------- 0/2 [distributed]
   -------------------- ------------------- 1/2 [bokeh]
   -------------------- ------------------- 1/2 [bokeh]
   -------------------- ------------------- 1/2 [bokeh]
   -------------------- ------------------- 1/2 [bokeh]
   -------------------- ------------------- 1/2 [bokeh]
   -------------------- ------------------- 1/2 [bokeh]
   -------------------- ------------------- 1/2 [bokeh]
   -------------------- ------------------- 1/2 [bokeh]
   -------------------- ------------------- 1/2 [bokeh]
   -

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report
import joblib

# -----------------------------
# 1️⃣ Load dataset in chunks and sample
# -----------------------------
data_path = r"D:\Development\yggdrasil\backend\ML model\synthetic_patient_data_new.csv"

# Read a sample of 200000 rows to avoid MemoryError
chunksize = 200000
sampled_chunks = []

for chunk in pd.read_csv(data_path, chunksize=chunksize):
    sampled_chunks.append(chunk.sample(frac=0.2, random_state=42))  # 20% of each chunk

df = pd.concat(sampled_chunks, ignore_index=True)

# -----------------------------
# Drop unique identifier
# -----------------------------
df = df.drop(columns=["Patient_Id"])

# -----------------------------
# Features and target
# -----------------------------
y = df["Risk_Level"]
X = df[["Age", "Gender", "Mood_Score", "Sleep_Quality", "Stress_Level", "Emotional_State"]]

# -----------------------------
# 2️⃣ Handle missing values
# -----------------------------
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns

num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

# -----------------------------
# 3️⃣ Downcast numeric types
# -----------------------------
X[num_cols] = X[num_cols].astype('float32')

# -----------------------------
# 4️⃣ Encode categorical features
# -----------------------------
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    encoders[col] = le

# -----------------------------
# 5️⃣ Split dataset
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# -----------------------------
# 6️⃣ Train model
# -----------------------------
model = HistGradientBoostingClassifier(
    max_iter=200,
    random_state=42,
    early_stopping=True
)
model.fit(X_train, y_train)

# -----------------------------
# 7️⃣ Evaluate model
# -----------------------------
y_pred = model.predict(X_test)
print("Classification Report:\n")
print(classification_report(y_test, y_pred))

# -----------------------------
# 8️⃣ Save model and encoders
# -----------------------------
joblib.dump(model, "risk_model_hgb.pkl")
joblib.dump(encoders, "encoders.pkl")
print("✅ Model and encoders saved successfully!")


C:\Users\Suresh Krishna\AppData\Local\Temp\ipykernel_35360\1640572285.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[num_cols] = num_imputer.fit_transform(X[num_cols])
C:\Users\Suresh Krishna\AppData\Local\Temp\ipykernel_35360\1640572285.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])
C:\Users\Suresh Krishna\AppData\Local\Temp\ipykernel_35360\1640572285.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

Classification Report:

              precision    recall  f1-score   support

    Critical       0.33      0.39      0.36       114
        High       0.88      0.78      0.82     25609
         Low       1.00      1.00      1.00    198329
      Medium       0.97      0.98      0.98    175948

    accuracy                           0.98    400000
   macro avg       0.79      0.79      0.79    400000
weighted avg       0.98      0.98      0.98    400000

✅ Model and encoders saved successfully!


In [1]:
import pandas as pd
import joblib
import numpy as np

# Load model and encoders
model = joblib.load(r"D:\Development\yggdrasil\backend\ML model\risk_model_hgb.pkl")
encoders = joblib.load(r"D:\Development\yggdrasil\backend\ML model\encoders.pkl")
print("✅ Model and encoders loaded successfully!")

# Test patient data (must match features used in training)
test_patient = {
    "Age": [10],
    "Gender": ["Male"],
    "Mood_Score": [2],
    "Sleep_Quality": [7],
    "Stress_Level": [2],
    "Emotional_State": ["Sad"]
}

df_new = pd.DataFrame(test_patient)

# Encode categorical features
for col, le in encoders.items():
    if col in df_new.columns:
        df_new[col] = df_new[col].astype(str)
        # Replace unseen categories with first known class
        df_new[col] = df_new[col].apply(lambda x: x if x in le.classes_ else le.classes_[0])
        df_new[col] = le.transform(df_new[col])

# Convert numeric columns to float32 for consistency
num_cols = ["Age", "Mood_Score", "Sleep_Quality", "Stress_Level"]
df_new[num_cols] = df_new[num_cols].astype("float32")

# Predict risk
prediction = model.predict(df_new)[0]

# Map risk labels
risk_mapping = {
    "Low": "Low Risk",
    "Moderate": "Moderate Risk",
    "High": "High Risk"
}
prediction_mapped = risk_mapping.get(prediction, prediction)

print(f"Predicted Risk Level for the patient: {prediction_mapped}")


✅ Model and encoders loaded successfully!
Predicted Risk Level for the patient: Critical
